# Part 1

In [ ]:
alphabet = {u'a',u'b',u'c',u'd',u'e',u'f',u'g',u'h',u'i',u'j',u'k',u'l',u'm',u'n',u'o',u'p',
           u'q',u'r',u's',u't',u'u',u'v',u'w',u'x',u'y',u'z'}

eyre=sc.textFile('/public/tbiswas2/csc261/spark/bronte-jane-eyre.txt')
words=eyre.flatMap(lambda x: x.split())
letters=words.flatMap(lambda x: list(x))
letters1 = letters.map(lambda x: (x.lower(),1))
letterCountX=letters1.reduceByKey(lambda v1, v2: v1+v2).filter(lambda x : x[0] in alphabet)
letterCountX=letterCountX.sortByKey()

letterCountL = letterCountX.collect()


%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

letterCountList = letterCountL
zippedletterCount = zip(*letterCountList)
 
letters = zippedletterCount[0]
y_pos = np.arange(len(letters))
letterCount = zippedletterCount[1]
 
plt.bar(y_pos, letterCount, align='center', alpha=0.5)
plt.xticks(y_pos, letters)
plt.ylabel('Count')
plt.title('Letter Frequency')
 
plt.show()

# Query 1

In [ ]:
storeDF.count()

# Query 5

In [ ]:
storeRestrict = storeDF.filter(storeDF.store_name == 'WEGMANS MARKETPLACE')
postransRestrict = postransDF.select('store_num','trans_num')
newtable = postransRestrict.join(storeRestrict, storeRestrict.store_num == postransRestrict.store_num)
newtable1 = newtable.select('trans_num')
newtable1.count()

# Query 9

In [ ]:
itemDFLB = itemDF.filter(itemDF.size_unit_desc == 'LB')
itemRDD = itemDFLB.select("item_unt_qty").rdd
maxweight = itemRDD.max()[0]
itemDFLBmax = itemDFLB.filter(itemDFLB.item_unt_qty == maxweight)
itemDFLBmax.sort('item_unt_qty',ascending=False).show()

# Query 13

In [ ]:
from pyspark.sql import functions as F

postrans12 = postransDF.select('store_num','net_sales','trans_num','trans_date')
salescount = postransDF.groupBy(['store_num','trans_date'])\
            .agg(F.sum('net_sales').alias('summation'))
saleStore = salescount.groupBy('store_num').max('summation')
saleStore.join(storeDF,'store_num').select('store_name','max(summation)')\
            .sort('max(summation)',ascending=False)\
            .show(3)

# Query 14

In [ ]:
from pyspark.sql import functions as F

transsalesitemsDF = postransDF.select('trans_num', 'net_sales','item_number')
transsalesDF = postransDF.select('trans_num','net_sales')
sumsalestransDF = transsalesDF.groupBy('trans_num').agg(F.sum('net_sales').alias('total_net_sales'))
transitemDF = transsalesitemsDF
sumssaletransDD = sumsalestransDF.sort('total_net_sales',ascending=False).rdd
ss11 = sumssaletransDD.zipWithIndex().filter(lambda (r,i): i < 3).map(lambda x: x[0])
dataframenew = spark.createDataFrame(ss11).select('trans_num')
rddnew1 = postransDF.join(dataframenew,'trans_num').select('trans_num','item_number').rdd
rddnew2 = rddnew1.map(lambda x: (x[0],[ x[1] ])).reduceByKey(lambda x,y: x+y)
rddnew2.collect()